In [1]:
import random
answer = random.randint(0, 100)
print("Let\'s guess the numbers! (0~100)")
guess_number = input("Please input you guess!\n")
score = 100
while(True):
    if guess_number.isdigit() == False:
        guess_number = input("Please input a number! Try again!\n")
        score -=10
        continue
    else:
        guess_number = int(guess_number)
    if guess_number > answer:
        score -=5
        guess_number = input("Your guess is too big. Try again!\n")
    elif guess_number < answer:
        score -=5
        guess_number = input("Your guess is too small. Try again!\n")
    else:
        print("Congratulations! You guessed it!\n" + "You score is" , score)
        if input("Play again? [1]yes [2]no\n") == "1":
            score = 100
            guess_number = input("Please input you guess!\n")
        else:
            break   

Let's guess the numbers! (0~100)
Please input you guess!
67
Your guess is too small. Try again!
80
Your guess is too small. Try again!
94
Your guess is too big. Try again!
86
Congratulations! You guessed it!
You score is 85
Play again? [1]yes [2]no
2


In [1]:
import time

count_down = 5# 设置倒计时时间，单位：秒
interval = count_down/50 # 设置屏幕刷新的间隔时间，单位：秒
for i in range(0, int(count_down/interval)+1):
    print("\r"+"▇"*i+" "+str(i*100/count_down*interval)+"%", end="")
    time.sleep(interval)

▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 100.0%

In [1]:
# -*- coding: utf-8 -*-
import base64
import pyperclip
from Crypto.Util.Padding import pad, unpad
from Crypto.Cipher import AES
from colorama import init, Fore
init(autoreset=True)


def pad_data(data):
    if len(data.encode('utf-8')) % AES.block_size != 0:
        data = pad(data.encode('utf-8'), AES.block_size, style='pkcs7')
    return data


def unpad_data(data):
    data = unpad(data, AES.block_size, style='pkcs7')
    return data.decode('utf-8')


def check_keyiv(key, iv):
    iv = iv.encode('utf-8')
    key = key.encode('utf-8')
    if len(key) != 16 and len(key) != 24 and len(key) != 32:
        return "error", f"[!] key({len(key)}b) is invalid"
    elif len(iv) != 16:
        return "error", f"[!] iv({len(iv)}b) is invalid"
    else:
        return key, iv


def encrypt(data, key, iv):
    check = check_keyiv(key, iv)
    if check[0] == "error":
        return check[1]
    else:
        data_padded = pad_data(data)
        aes = AES.new(check[0], AES.MODE_CBC, check[1])
        encryption = aes.encrypt(data_padded)
        return base64.b64encode(encryption).decode("utf-8")


def decrypt(data, key, iv):
    check = check_keyiv(key, iv)
    if check[0] == "error":
        return check[1]
    else:
        data_unb64 = base64.b64decode(data.encode("utf-8"))
        aes = AES.new(check[0], AES.MODE_CBC, check[1])
        decryption = aes.decrypt(data_unb64)
        return unpad_data(decryption)


def main():
    key_dict = {
        "key_orig": "ldaldkfmcnvifkdh",
        "key_21": "wqedycgafbnmpoqw",
        "key_S1": "m2zxwec3hslwie7r",
        "key_S2": "sldh2we7er9weuyf"
    }
    key = key_dict["key_orig"]  # original key
    iv = 'slkoiewlkduioakd'
    print("AES encryption program")
    print("Commands:")
    print("[1]: Encrypt\n[2]: Decrypt\n[3]: Change keys")
    print("[help]: Get more information")
    while (True):
        choice = input()
        if choice == "1":
            text = input("Plaintext:")
            ciphertext = encrypt(text, key, iv)
            check_encrypt = decrypt(ciphertext, key, iv)
            print("Ciphertext:" + Fore.GREEN + ciphertext)
            print(f"check:{check_encrypt}")
            if check_encrypt == text:
                print("successful!")
                pyperclip.copy(ciphertext)
        elif choice == "2":
            text = input("Ciphertext:")
            try:
                plaintext = decrypt(text, key, iv)
            except Exception as e:
                print(Fore.RED + "[!] Decryption error")
            else:
                check_decrypt=encrypt(plaintext, key, iv)
                print("Plaintext:" + Fore.GREEN + plaintext)
                print(f"check:{check_decrypt}")
                if check_decrypt == text:
                    print("successful!")
                    pyperclip.copy(plaintext)
        elif choice == "3":
            while (True):
                key_new=input("key(16/24/32b):")
                if key_dict.get(key_new) is not None:
                    key=key_dict.get(key_new)
                    print(f"Successfully replaced the existing[{key_new}]")
                    break
                elif check_keyiv(key_new, iv)[0] != "error":
                    key=key_new
                    print("Key changed successfully")
                    break
                elif len(key_new) == 0:
                    print("[*] Cancel change key")
                    break
                else:
                    print(Fore.RED + check_keyiv(key_new, iv)[1])
        elif choice == "help":
            print(
                "\nAES(128/192/256):Advanced Encryption Standard\n\nMode:CBC")
            print("Pad:pkcs7padding\nOutput:base64\n\nMade by Ymx Studio")


if __name__ == '__main__':
    main()

ModuleNotFoundError: No module named 'pyperclip'